In [1]:
#puts above directory into the path
import sys
sys.path.append("..")
import vrep

In [2]:
import numpy as np
import time

In [3]:
class VREP_Environement():
    ''' This object defines a VREP environment '''
    def __init__(self, synchronous = False, dt = 0.05):
        self.dt = dt
        self.synchronous = synchronous
        self.robots_connected = 0
        self.robot_names = []
        self.handles_init = None
        self.clientID = None

        #Close any open connections
        vrep.simxFinish(-1)
        
        #Initiate connection to server
        self.connectToServer()

    def connectToServer(self):
        self.clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 500, 5)
        
        if self.clientID != -1: # if we connected successfully
            print ('Connected to remote API server')
            
        #Setup synchronous mode or not
        if self.synchronous == True:
            print("In synchronous mode")
            vrep.simxSynchronous(self.clientID, True)

    def add_robot(self, robot_class):
        self.robots_connected = self.robots_connected + 1
        
        #Vrep env class stuff
        self.robot_names.append(robot_class.robot_name)
        
        #robot_class stuff
        robot_class.clientID = self.clientID
        robot_class.get_handles()
        
        #Add robot to class attributes
        setattr(self, robot_class.robot_name, robot_class)
        
    def start_simulation(self):
        if self.robots_connected == 0:
            print("no robots connected, simulation not started")
        else:
            # Set up streaming
            print("{} robot(s) connect: {}".format(self.robots_connected, self.robot_names))
            vrep.simxSetFloatingParameter(
                self.clientID,
                vrep.sim_floatparam_simulation_time_step,
                self.dt, # specify a simulation time stept
                vrep.simx_opmode_oneshot)
        
            # Start the simulation
            #vrep.simxStartSimulation(self.clientID,vrep.simx_opmode_blocking) #to increase loop speed mode is changed.
            vrep.simxStartSimulation(self.clientID,vrep.simx_opmode_oneshot_wait)


    def shutdown(self):
        vrep.simxStopSimulation(self.clientID, vrep.simx_opmode_oneshot)
        vrep.simxFinish(self.clientID)


In [4]:
class VREP_Robot():
    '''This object defines the robots in the environment'''
    def __init__(self, robot_name, handle_names, connection_type = 'nonblocking'):
        self.robot_name = robot_name
        self.handle_names = handle_names
        self.handles = None
        self.num_poses = len(handle_names) 
        self.positions = np.ones([self.num_poses,3]) #xyz
        self.orientations = np.ones([self.num_poses,4]) #xyzw
        self.connection_type = connection_type
        self.clientID = None
        
        #Connection type for object grabbing/setting
        if self.connection_type == 'blocking':
            self.opmode = vrep.simx_opmode_blocking
        elif self.connection_type == 'nonblocking':
            self.opmode = vrep.simx_opmode_oneshot
        
    def get_handles(self):
        self.handles = [vrep.simxGetObjectHandle(self.clientID,
            name, vrep.simx_opmode_blocking)[1] for name in self.handle_names]
        
    def setObjectPosition(self, object_name, cartesian_position):
        if self.clientID == None:
            print("Robot not attached to VREP environment")
        else:
            vrep.simxSetObjectPosition(
                self.clientID,
                object_name,
                -1,# Setting the absolute position
                position=cartesian_position,
                operationMode=self.opmode
                )
            
    def setObjectQuaternion(self, object_name, quaternion):
        if self.clientID == None:
            print("Robot not attached to VREP environment")
        else:
            vrep.simxSetObjectQuaternion(
                self.clientID,
                object_name,
                vrep.sim_handle_parent,# Setting the absolute position
                quaternion, #(x, y, z, w)
                operationMode = self.opmode
                )

    def getObjectPosition(self, object_name):
        if self.clientID == None:
            print("Robot not attached to VREP environment")
        else:
            cartesian_position = vrep.simxGetObjectPosition(
                self.clientID,
                object_name,
                -1,# Setting the absolute position
                operationMode = self.opmode
                )
        return cartesian_position
    
    def getObjectOrientation(self, object_name):
        if self.clientID == None:
            print("Robot not attached to VREP environment")
        else:
            quaternion = vrep.simxGetObjectQuaternion(
                self.clientID,
                object_name,
                -1,# Setting the absolute position
                operationMode = self.opmode
                )
        return quaternion

In [5]:
# http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm#simxSetObjectQuaternion

#To use classes above, open vrep scene (robot.ttt) first then you can run code below

# Availalble commands 
# vrep_env.robot.getObjectPosition
# vrep_env.robot.getObjectOrientation
# vrep_env.robot.setObjectPosition
# vrep_env.robot.setObjectQuaternion
# vrep_env.robot.positions #xyz all positions in robot, dim(num joints/frames, 3)
# vrep_env.robot.orientations #xyzw quaternion i think, all orientations in robot, dim(num joints/frames, 4)
# vrep_env.robot.connection_type
# vrep_env.robot.handle_names
# vrep_env.add_robot() - is used to add robots to the environment, must be done before starting simulation
# vrep_env.start_simulation()
# vrep_env.shutdown() #not working right now

# Each command to the vrep server requires an operation mode (blocking, oneshot, etc...), for setting things up
# use blocking mode. This makes sure things are connected etc before moving forward. Things like getting/setting information
# can be used in async mode. This is vrep_env.robot.connection_type.

In [53]:
vrep_env = VREP_Environement()

Connected to remote API server


In [54]:
#robot_handles = ['pose_j0', 'pose_j1']
#vrep_env.add_robot(VREP_Robot('ik_robot', robot_handles))

robot_handles  = ['pose_j0', 'pose_j1', 'pose_j2', 'pose_j3', 'pose_j4', 'pose_j5', 'pose_j6', 'pose_j7']
vrep_env.add_robot(VREP_Robot('robot', robot_handles))

# robot_handles  = ['joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6', 'joint7']
# vrep_env.add_robot(VREP_Robot('robot', robot_handles))

# ik_handles = ['ik_joint1', 'ik_joint2', 'ik_joint3', 'ik_joint4', 'ik_joint5', 'ik_joint6', 'ik_joint7','kinematicsTest_manipulatorSphere']
# vrep_env.add_robot(VREP_Robot('ik_robot', ik_handles))

In [55]:
#Add all robot before starting the simulation - once robots are added start simulation
vrep_env.start_simulation()

1 robot(s) connect: ['robot']


In [58]:
print(vrep_env.robot.handle_names)
for i in range(len(robot_handles)):
    vrep_env.robot.setObjectPosition(vrep_env.robot.handles[i], [0])

['pose_j0', 'pose_j1', 'pose_j2', 'pose_j3', 'pose_j4', 'pose_j5', 'pose_j6', 'pose_j7']


In [59]:
for i in range(len(robot_handles)):
    print(vrep_env.robot.getObjectPosition(vrep_env.robot.handles[i]))

(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])
(0, [0.0, 0.0, 0.0])


In [ ]:
print(vrep_env.ik_robot.handle_names)

In [ ]:
for i in range(len(ik_handles)):
    print(vrep_env.ik_robot.getObjectPosition(vrep_env.ik_robot.handles[i]))

In [ ]:
for i in range(len(ik_handles)):
    print(vrep_env.ik_robot.getObjectOrientation(vrep_env.ik_robot.handles[i])[1])

In [ ]:
vrep_env.ik_robot.setObjectPosition(vrep_env.ik_robot.handles[-1], [.1,0,0])

In [ ]:
for i in range(len(robot_handles)):
    vrep_env.robot.setObjectQuaternion(vrep_env.robot.handles[i],vrep_env.ik_robot.getObjectOrientation(vrep_env.ik_robot.handles[i])[1])

In [ ]:
vrep_env.shutdown() #not working right now

In [ ]:
EE = vrep.simxGetObjectHandle(clientID, 'kinematicsTest_manipulatorSphere', vrep.simx_opmode_blocking)
current_pos = vrep.simxGetObjectPosition(clientID, EE[1], -1, vrep.simx_opmode_streaming)[1]

In [ ]:
vrep.simxFinish(-1)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 500, 5) 
if clientID != -1: # if we connected successfully
        print ('Connected to remote API server')

In [ ]:
EE = vrep.simxGetObjectHandle(clientID, 'kinematicsTest_manipulatorSphere', vrep.simx_opmode_blocking)

In [ ]:
EE

In [ ]:
dt = .05
vrep.simxSetFloatingParameter(clientID,
        vrep.sim_floatparam_simulation_time_step,
        dt, # specify a simulation time step
        vrep.simx_opmode_oneshot)
 
# --------------------- Start the simulation
# start our simulation
vrep.simxStartSimulation(clientID,vrep.simx_opmode_oneshot_wait)

In [ ]:
current_pos = vrep.simxGetObjectPosition(clientID, EE[1], -1, vrep.simx_opmode_streaming)[1]

In [ ]:
current_pos

In [ ]:
start_time = time.time()
current_pos = np.array(current_pos)
while(time.time() - start_time <= 15):
    new_pos = current_pos + 0.2*np.sin(time.time()*2)
    vrep.simxSetObjectPosition(clientID, EE[1], -1, new_pos ,vrep.simx_opmode_oneshot)
    print(vrep.simxGetObjectPosition(clientID, EE[1], -1, vrep.simx_opmode_streaming)[1])
    time.sleep(2)
    